# Sentence Transformers

In [1]:
import numpy as np
import pandas as pd
import os

# weather sentences (utils.py)

In [2]:
!pip install timezonefinder
#!pip install geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00


In [128]:
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from datetime import datetime, timedelta
import pandas as pd


def get_coords_from_location_name(location: str = "Berlin, Germany"):
    """
    Translates place name like "Berlin, Germany" into (latitude, longitude)
    """
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(location)
    if location is None:
        print("Location could not be resolved. Using 'Berlin, Germany' instead.")
        location = geolocator.geocode("Berlin, Germany")
    return (location.latitude, location.longitude)


def get_timezone_from_coords(latitude, longitude):
    tf = TimezoneFinder()
    tz_name = tf.timezone_at(lat=latitude, lng=longitude)
    return tz_name


def describe_by_threshold(value, thresholds):
    """Return the first matching description based on threshold values."""
    for limit, desc in thresholds:
        if value < limit:
            return desc
    return thresholds[-1][1]  # default (last) description


def describe_weather(weather_info: dict):
    time = weather_info["time"].hour
    hum = weather_info["humidity"]
    humid = describe_by_threshold(
        hum,
        [
            (30, "extremely dry"),
            (40, "rather dry"),
            (60, "neither dry nor humid"),
            (70, "humid"),
            (85, "pretty humid"),
            (float("inf"), "moist"),
        ],
    )

    temp = weather_info["temperature"]
    warmth = describe_by_threshold(
        temp,
        [
            # (-10, "insufferably cold, way below freezing"),
            # (0, "below freezing"),
            # (10, "rather cold and uncomfortable"),
            # (15, "cool and fresh"),
            # (18, "fresh and slightly uncomfortable"),
            # (22, "mild and pleasant"),
            # (26, "warm"),
            # (30, "very warm, almost hot"),
            # (float("inf"), "hot and unpleasant"),
            (1, "freezing"),
            (10, "cold"),
            (15, "cool and fresh"),
            (20, "mild and pleasant"),
            (25, "warm"),
            (30, "hot"),
            (float("inf"), "hot and unpleasant"),
        ],
    )

    wind = weather_info["wind"]
    windy = describe_by_threshold(
        wind,
        [
            (0, "perfectly windstill"),
            (5, "very calm"),
            (10, "slightly breezy"),
            (20, "breezy and gusty"),
            (30, "stormy and very gusty"),
            (float("inf"), "the time of a storm"),
        ],
    )

    rain = weather_info["rain"]
    raining = describe_by_threshold(
        rain,
        [
            (1, "not raining"),
            (3, "slightly drizzling"),
            (5, "raining"),
            (10, "raining heavily"),
            (float("inf"), "raining cats and dogs"),
        ],
    )

    sn = f"Today it will be {warmth} and {windy}."
    sn += f"The air will feel {humid} and it will be {raining}."

    return sn


In [131]:
input = {
        "time": [datetime.now() + timedelta(hours=i) for i in range(6)],
        "humidity": [80, 50, 50, 50, 60, 70],
        "temperature": [0, 11, 12, 12, 13, 13],
        "wind": [5, 16, 18, 20, 22, 30],
        "rain": [4, 0, 0, 0, 1.0, 1.5],
    }
df = pd.DataFrame(input)  # replace with weather_prediction
weather_sentence = describe_weather(df.iloc[0])

In [105]:
describe_weather(df.iloc[5])

'Today it will be cool and fresh and the time of a storm.The air will feel pretty humid and it will be slightly drizzling.'

In [163]:
weather_sentence

'Today it will be freezing and slightly breezy.The air will feel pretty humid and it will be raining.'

# sample clothing df

In [7]:
from google.colab import files
uploaded = files.upload()

Saving new_classified_top.csv to new_classified_top.csv


In [8]:
top_df = pd.read_csv("new_classified_top.csv")
top_df.head()

,product_id,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
0,5,[{'https://static.zara.net/photos///2022/V/2/1...,women,TOPS_BODYSUITS,kabuki brush,short smooth densebristle brush ideal apply fo...,Top,other,short,other,short,short short,This is a kabuki brush made of other with shor...,mild weather
1,12,[],women,JACKETS,coat nail varnish,quickdrye coat highlight colour nail ensure du...,Top,other,coat,other,other,coat,This is a coat nail varnish made of other with...,warm weather
2,13,[],women,TOPS_BODYSUITS,nail polish,superior wear naturally highperformance formul...,Top,other,other,other,long,long,This is a nail polish made of other with long ...,mild weather
3,14,[],women,TOPS_BODYSUITS,find,find,Top,other,other,other,other,NaN,This is a find made of other with other coverage.,mild weather
4,16,[],women,TOPS_BODYSUITS,concealer,creamy luscious formula easy blend buildable m...,Top,satin,other,other,other,satin,This is a concealer made of satin with other c...,mild weather


# try out embedding and cosine-similarity with all-MiniLM-L6-v2

In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [181]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_DISABLED"] = "true"

# 1. Start with a base model
model = SentenceTransformer('all-MiniLM-L6-v2')
# 2. Prepare labeled data → pairs of (text, label)
examples = [
    ("-2°C snowy day", "wool coat with thermal lining", "linen shorts"),
    ("10°C windy morning", "denim jacket with long sleeves", "tank top"),
    ("18°C cloudy afternoon", "cotton long-sleeve shirt", "puffer jacket"),
    ("25°C sunny weather", "light cotton t-shirt", "wool sweater"),
    ("30°C hot summer day", "linen dress", "heavy denim jacket"),
    ("15°C rainy evening", "waterproof raincoat", "shorts")
    ]

train_examples = [
    InputExample(texts=[anchor, positive, negative])
    for anchor, positive, negative in examples
]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=3)

# 3. Use Triplet loss
#train_loss = losses.CosineSimilarityLoss(model)
train_loss = losses.TripletLoss(model=model)

# 4. Train briefly (it's lightweight)
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=200,
    warmup_steps=10,
    show_progress_bar=True,
    output_path=None
)
# 5. Save your fine-tuned model
model.save('weather_clothing_miniLM')

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


In [182]:
model = model.load('weather_clothing_miniLM')

clothes_text = top_df['text_complex'] + ' ' + top_df['weather_label']
clothes_labels = clothes_text.tolist()
weather_outfit_description = weather_sentence
weather_outfit_description

/tmp/ipython-input-908419498.py:1: DeprecationWarning: SentenceTransformer.load(...) is deprecated, use SentenceTransformer(...) instead.
  model = model.load('weather_clothing_miniLM')


'Today it will be freezing and slightly breezy.The air will feel pretty humid and it will be raining.'

In [183]:
clothing_embeds = model.encode(clothes_labels, normalize_embeddings=True)
weather_embed = model.encode(weather_outfit_description, normalize_embeddings=True)

scores = cosine_similarity([weather_embed], clothing_embeds)[0]
scores

array([ 0.48211446,  0.4349132 ,  0.28920758, ..., -0.15366651,
       -0.03313942, -0.5523109 ], dtype=float32)

In [184]:
ix = np.argsort(scores)[-1]
scores[ix], clothes_labels[ix]

(np.float32(0.9560238),
 'This is a bikini frill made of other with other coverage. warm weather')

In [185]:
ix = np.argsort(scores)[-2]
scores[ix], clothes_labels[ix]

(np.float32(0.9482775),
 'This is a mickey mouse disney sweatshirt made of other with short coverage. cool weather')

In [186]:
ix = np.argsort(scores)[-3]
scores[ix], clothes_labels[ix]

(np.float32(0.9482775),
 'This is a mickey mouse disney sweatshirt made of other with short coverage. cool weather')

In [16]:
np.argsort(scores)[-3:]

array([1017, 1015, 1361])

# try out embedding and cosine-similarity with all-mpnet-base-v2

In [139]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

clothes_text = top_df['text_complex'] + ' ' + top_df['weather_label']
clothes_labels = clothes_text.tolist()
weather_outfit_description = weather_sentence
weather_outfit_description

'Today it will be freezing and slightly breezy.The air will feel pretty humid and it will be raining.'

In [140]:
clothing_embeds = model.encode(clothes_labels, normalize_embeddings=True)
weather_embed = model.encode(weather_outfit_description, normalize_embeddings=True)

scores = cosine_similarity([weather_embed], clothing_embeds)[0]
scores

array([0.09616837, 0.1770583 , 0.303217  , ..., 0.26117796, 0.25754774,
       0.3431806 ], dtype=float32)

In [141]:
ix = np.argsort(scores)[-1]
scores[ix], clothes_labels[ix]

(np.float32(0.45123595),
 'This is a rip fadedeffect sweatshirt made of cotton with long coverage. It is suitable for mild or cool weather. mild weather')

In [142]:
ix = np.argsort(scores)[-2]
scores[ix], clothes_labels[ix]

(np.float32(0.45123595),
 'This is a rip fadedeffect sweatshirt made of cotton with long coverage. It is suitable for mild or cool weather. mild weather')

In [143]:
ix = np.argsort(scores)[-3]
scores[ix], clothes_labels[ix]

(np.float32(0.44982556),
 'This is a polo sweatshirt made of cotton with long coverage. It is suitable for mild or cool weather. mild weather')

In [51]:
np.argsort(scores)[-3:]

array([1646, 1373, 1644])